In [87]:
import os
import pickle
import numpy as np
import datetime
import json
from multiprocessing import Process
from multiprocess import Process

import keras
from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications import VGG16, MobileNetV2, MobileNetV3Small

from models import *
from utils import regression_stats
from img_utils import data_to_df, preprocess_images, set_gpu, set_cpu

import matplotlib.pyplot as plt

In [56]:
from img_utils import data_to_df
#try reloading the module
IMAGE_WIDTH: int = 448
IMAGE_HEIGHT: int = 448


# Load Data
train_1_path: str = "../nybolig-scrape/output/train/train_1"
train_2_path: str = "../nybolig-scrape/output/train/train_2"
valid_path: str = "../nybolig-scrape/output/valid"
test_path: str = "../nybolig-scrape/output/test"

train1_df, train2_df, valid_df, test_df = data_to_df(
    [train_1_path, train_2_path, valid_path, test_path], preprocess=True
)

# TODO: Would be better with this format
# train_images: np.array = preprocess_images(
#     train1_df["image_floorplan"], IMAGE_WIDTH, IMAGE_HEIGHT, True, False, False
# )


#### Train Set 1 ####
train1_features = train1_df.drop(columns=["image_floorplan", "price"])
train1_images: np.array = preprocess_images(
    train1_df, "image_floorplan", IMAGE_WIDTH, IMAGE_HEIGHT, True, False, False
)
train1_prices: np.array = train1_df["price"].values


#### Train Set 2 ####
train2_features = train2_df.drop(columns=["image_floorplan", "price"])
train2_images: np.array = preprocess_images(
    train2_df, "image_floorplan", IMAGE_WIDTH, IMAGE_HEIGHT, True, False, False
)
train2_prices: np.array = train2_df["price"].values


#### Validation Set ####
valid_features = valid_df.drop(columns=["image_floorplan", "price"])
valid_images: np.array = preprocess_images(
    valid_df, "image_floorplan", IMAGE_WIDTH, IMAGE_HEIGHT, True, False, False
)
valid_prices: np.array = valid_df["price"].values


#### Test Set ####
test_features = test_df.drop(columns=["image_floorplan", "price"])
test_images: np.array = preprocess_images(
    test_df, "image_floorplan", IMAGE_WIDTH, IMAGE_HEIGHT, True, False, False
)
test_prices: np.array = test_df["price"].values

Processing ../nybolig-scrape/output/train/train_1: 100%|██████████| 113/113 [00:00<00:00, 142714.95it/s]
Processing ../nybolig-scrape/output/train/train_2: 100%|██████████| 114/114 [00:00<00:00, 305722.93it/s]
Preprocessing: 100%|██████████| 4/4 [00:00<00:00, 28.13it/s]


In [90]:
def save_expected_predicted(test_prices, test_predictions, img_dir):
        #Set X and Y axis to [0, 9.000.000]
    #plt.xlim(0, 9999999)
    #plt.ylim(0, 9999999)
    plt.scatter(test_prices, test_predictions)
    plt.xlabel("Expected Price")
    plt.ylabel("Predicted Price")
    plt.title("Expected vs Predicted Price")
    try: 
        plt.plot([min(test_prices), max(test_prices)], [min(test_prices), max(test_prices)], color='red')
    except:
        pass
    plt.savefig(f"{img_dir}/expected_vs_predicted.png")
    plt.close()

def save_residuals(test_prices, test_predictions, img_dir):
    residuals = test_prices - test_predictions.reshape(-1)
    plt.scatter(test_predictions, residuals)
    try:
        plt.hlines(y=0, xmin=test_prices.min(), xmax=test_prices.max(), colors="r")
    except:
        pass
    plt.xlabel("Expected Price")
    plt.ylabel("Residuals")
    plt.title("Residuals")
    plt.savefig(f"{img_dir}/residuals.png")
    plt.close()

def get_saliency_map(model, image):
    image = np.expand_dims(image, axis=0)
    image = image / 255.0
    image = image.astype(np.float32)
    image = tf.convert_to_tensor(image)
    with tf.GradientTape() as tape:
        tape.watch(image)
        prediction = model(image)
    gradients = tape.gradient(prediction, image)
    gradients = tf.squeeze(gradients)
    gradients = tf.reduce_max(gradients, axis=-1)
    gradients = gradients.numpy()
    gradients = (gradients - np.min(gradients)) / (np.max(gradients) - np.min(gradients))
    return gradients

def save_worst_best_predictions(model, test_predictions, test_prices, test_images, img_dir):
    residuals = test_prices - test_predictions.reshape(-1)
    distances = np.abs(test_prices - test_predictions.reshape(-1))
    worst_predictions = np.argsort(distances)[-8:]
    best_predictions = np.argsort(distances)[:8]
    test_images = np.array(test_images)
    for i, idx in enumerate(worst_predictions):
        image = test_images[idx]
        price = test_prices[idx]
        prediction = test_predictions[idx]
        residual = residuals[idx]
        plt.imshow(image)
        textstr = '\n'.join((
            f"Price: {price}",
            f"Predicted Price: {prediction}",
            f"Residual: {residual}"
        ))
        plt.text(0.01, 0.99, textstr, fontsize=10, transform=plt.gcf().transFigure, verticalalignment='top')
        plt.axis("off")
        plt.savefig(f"{img_dir}/worst_{i}.png")
        plt.close()
        
        saliency_map = get_saliency_map(model, image)
        plt.imshow(saliency_map, cmap="hot")
        plt.axis("off")
        plt.savefig(f"{img_dir}/worst_saliency_map_{i}.png")
        plt.close()
        
    for i, idx in enumerate(best_predictions):
        image = test_images[idx]
        price = test_prices[idx]
        prediction = test_predictions[idx]
        residual = residuals[idx]
        plt.imshow(image)
        textstr = '\n'.join((
            f"Price: {price}",
            f"Predicted Price: {prediction}",
            f"Residual: {residual}"
        ))
        plt.text(0.01, 0.99, textstr, fontsize=10, transform=plt.gcf().transFigure, verticalalignment='top')
        plt.axis("off")
        plt.savefig(f"{img_dir}/best_{i}.png")
        plt.close()
        saliency_map = get_saliency_map(model, image)
        plt.imshow(saliency_map, cmap="hot")
        plt.axis("off")
        plt.savefig(f"{img_dir}/best_saliency_map_{i}.png")
        plt.close()

def save_features_importance(feature_importance, img_dir):
    #sort the feature_importance dict by value
    feature_importance = {k: v for k, v in sorted(feature_importance.items(), key=lambda item: item[1], reverse=True)}
    #add percentages to the bars
    plt.bar(feature_importance.keys(), feature_importance.values())
    #plt.bar_label = feature_importance.values()
    plt.title('Feature Importance')
    #Remove y-labels
    plt.ylabel('')
    plt.xticks(rotation=90)
    #Zoom out so that text is visible 
    plt.subplots_adjust(bottom=0.4)
    plt.savefig(f"{img_dir}/feature_importance.png")
    plt.close()

def save_worst_best(test_predictions, test_prices, test_features, model_dir):
    #Find the best predictions, and worst predictions. 
    #Save them in two dataframes. Save a latex of the dataframe in a txt-file 
    residuals = test_prices - test_predictions.reshape(-1)
    distances = np.abs(test_prices - test_predictions.reshape(-1))
    worst_predictions = np.argsort(distances)[-8:]
    best_predictions = np.argsort(distances)[:8]
    
    test_features_ = pd.DataFrame(test_features).copy()
    test_features_["Price"] = test_prices
    test_features_["Predicted Price"] = test_predictions
    test_features_["Residual"] = residuals
    test_features_ = test_features.sort_values(by="Residual", ascending=False)
    worst_df = test_features_.iloc[worst_predictions]
    best_df = test_features_.iloc[best_predictions]
    #save worst and best as latex in txt-file 
    worst_df.to_latex(f"{model_dir}/worst_predictions.txt")
    best_df.to_latex(f"{model_dir}/best_predictions.txt")
    


def save_model_and_evaluate(
    model: object,
    fit_history: object,
    test_images: np.array,
    test_features: np.array,
    test_prices: np.array,
    model_dir: str,
    model_type:str
):
    if model_type == 'RF':
        print("Saving Model...")
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
        with open(f"{model_dir}/model", "wb") as file_pi:
            pickle.dump(model, file_pi)
        test_predictions = model.predict(test_features)
    
    if model_type == "CNN":
        # Save Model
        print("Saving Model...")
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
        model.save(f"{model_dir}/model")
        # Save Training History
        with open(f"{model_dir}/history", "wb") as file_pi:
            pickle.dump(fit_history.history, file_pi)
        test_predictions = model.predict(test_images)
        #Save Model Architecture
        #plot_model(model, to_file=f"{model_dir}/model_architecture.png", show_shapes=True, show_layer_names=True, show_dtype=True, rankdir="TB", expand_nested=False, dpi=96)
        img = plot_model(model, to_file=f"{model_dir}/architecture.png", show_shapes=True, show_layer_names=True, show_dtype=True, rankdir="TB", expand_nested=False, dpi=96)


    if model_type == 'CNN_RF':
        print("Saving Model...")
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
        with open(f"{model_dir}/model", "wb") as file_pi:
            pickle.dump(model, file_pi)
        test_predictions = model.predict(test_images, test_features)
        

    # Evaluate Model
    print("Evaluating Model...")
    r2, mae, percentage_error, mse = regression_stats(test_prices, test_predictions)

    try:
        feature_importance = model.feature_importances_
        if model_type == "RF":
            feature_importance = dict(zip(test_features.columns, feature_importance))
    except AttributeError:
        print("Cant find feature_importance")
        feature_importance = None

    # Load existing evaluation data
    evaluation_file_path = f"{model_dir}/evaluation.json"
    evaluation_data = {}
    if os.path.exists(evaluation_file_path):
        with open(evaluation_file_path, "r") as json_file:
            evaluation_data = json.load(json_file)

    # Add new evaluation data
    new_evaluation = {
        "Timestamp": str(datetime.datetime.now()),
        "R2": r2,
        "MAE": mae,
        "Percentage Error": percentage_error,
        "MSE": mse,
        "Feature Importances": (feature_importance),
    }
    evaluation_data[len(evaluation_data)] = new_evaluation

    # Save updated evaluation data
    with open(evaluation_file_path, "w") as json_file:
        json.dump(evaluation_data, json_file, indent=4)

    # Compute median evaluation values from all instances
    r2_values = [evaluation_data[key]["R2"] for key in evaluation_data]
    mae_values = [evaluation_data[key]["MAE"] for key in evaluation_data]
    percentage_error_values = [
        evaluation_data[key]["Percentage Error"] for key in evaluation_data
    ]
    mse_values = [evaluation_data[key]["MSE"] for key in evaluation_data]

    median_evaluation_data = {
        "R2": np.median(r2_values),
        "MAE": np.median(mae_values),
        "Percentage Error": np.median(percentage_error_values),
        "MSE": np.median(mse_values),
    }

    with open(f"{model_dir}/median_evaluation.json", "w") as json_file:
        json.dump(median_evaluation_data, json_file, indent=4)

    print("\nModel Evaluation:")
    print(new_evaluation)
    print("\nMedian Evaluation:")
    print(median_evaluation_data)
    print("Feauter Importance...")
    print(feature_importance)

    # Images (Create or open existing folder)
    if not os.path.exists(f"{model_dir}/images"):
        os.makedirs(f"{model_dir}/images")
    img_dir = f"{model_dir}/images"
    
    save_expected_predicted(test_prices, test_predictions, img_dir)
    save_residuals(test_prices, test_predictions, img_dir)
    
    if model_type == 'CNN':
        print("\nSaving Best and Worst Image Predictions")
        save_worst_best_predictions(model, test_predictions, test_prices, test_images, img_dir)
    
    if model_type != 'CNN': 
        print("\nSaving Feature Importance")
        save_features_importance(feature_importance, img_dir)

    save_worst_best(test_predictions, test_prices, test_features, model_dir)
    print("\nDone!")




def train_save_model(
    model_func: object,
    args: tuple,
    test_images: np.array,
    test_features: np.array,
    test_prices: np.array,
    model_dir: str,
    use_gpu: bool,
    model_type:str
):
    if use_gpu:
        set_gpu()
    else:
        set_cpu()

    if model_type == "CNN":
        model, fit_history = model_func(*args)
    if model_type == 'RF':
        model = model_func(*args)
        fit_history = None
    if model_type == 'CNN_RF':
        model = model_func(*args)
        fit_history = None
    save_model_and_evaluate(model, fit_history, test_images, test_features, test_prices, model_dir, model_type)


def train_save_models(
    model_func: object,
    args: tuple,
    test_images: np.array,
    test_prices: np.array,
    model_dir: str,
    use_gpu: bool,
):
    if use_gpu:
        set_gpu()
    else:
        set_cpu()

    models, fit_histories = model_func(*args)
    for model_idx, (model, fit_history) in enumerate(zip(models, fit_histories)):
        save_model_and_evaluate(
            model, fit_history, test_images, test_prices, f"{model_dir}_{model_idx}"
        )

# Running on GPU

In [4]:
MODELS_PATH: str = "./models"
USE_GPU: bool = True

#TYPE = "RF"
# MODEL_NAME: str = "RF"
# FUNCTION: object = RF
# ARGS: tuple = (
#     train_images,
#     train_prices,
#     valid_images,
#     valid_prices,
# )
# p = Process(
#     target=train_save_model,
#     args=(FUNCTION, ARGS, test_images, test_prices, f"{MODELS_PATH}/{MODEL_NAME}", USE_GPU),
# )

TYPE = "CNN"
MODEL_NAME: str = "MobileNetV2"
FUNCTION: object = CNN_model
ARGS: tuple = (
    MobileNetV2,
    True,
    train1_images,
    train1_prices,
    valid_images,
    valid_prices,
)
p = Process(
    target=train_save_model,
    args=(FUNCTION, ARGS, test_images, test_prices, f"{MODELS_PATH}/{MODEL_NAME}", USE_GPU, TYPE),
)

#TYPE = "N_CNN"
# MODEL_NAME: str = "N_CNN_MobileNetV2"
# FUNCTION: object = N_CNN_model
# ARGS: tuple = (
#     MobileNetV2,
#     train_images,
#     train_prices,
#     valid_images,
#     valid_prices,
#     3,
# )
# p = Process(
#     target=train_save_models,
#     args=(FUNCTION, ARGS, test_images, test_prices, f"{MODELS_PATH}/{MODEL_NAME}"),
# )


p.start()
p.join()

2024-04-22 11:58:56.197612: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-22 11:58:56.238691: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-22 11:58:56.238782: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-22 11:58:56.242355: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-22 11:58:56.242558: I external/local_xla/xla/stream_executor

1 Physical GPUs, 1 Logical GPU


2024-04-22 11:58:56.411413: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-22 11:58:56.411591: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-22 11:58:56.411609: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2022] Could not identify NUMA node of platform GPU id 0, defaulting to 0.  Your kernel may not have been built with NUMA support.
2024-04-22 11:58:56.411670: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-22 11:58:56.411689: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created

Epoch 1/150


2024-04-22 11:59:03.690523: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-04-22 11:59:03.792601: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-22 11:59:04.024400: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-22 11:59:07.397578: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fe2fda5e1d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-22 11:59:07.397618: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1080, Compute Capability 6.1
2024-04-22 11:59:07.403020: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1713779947.498196   78260 device_compiler.

10/10 [==============================] - 16s 738ms/step - loss: 4099945.2500 - mean_absolute_error: 4098415.2500 - val_loss: 4399402.5000 - val_mean_absolute_error: 4398138.5000
Epoch 2/150
10/10 [==============================] - 3s 272ms/step - loss: 4072331.0000 - mean_absolute_error: 4070577.0000 - val_loss: 4310734.0000 - val_mean_absolute_error: 4307788.0000
Epoch 3/150
10/10 [==============================] - 3s 275ms/step - loss: 3848637.2500 - mean_absolute_error: 3844325.2500 - val_loss: 3719758.0000 - val_mean_absolute_error: 3712978.0000
Epoch 4/150
10/10 [==============================] - 3s 284ms/step - loss: 2681056.2500 - mean_absolute_error: 2672059.0000 - val_loss: 1882596.0000 - val_mean_absolute_error: 1870154.5000
Epoch 5/150
10/10 [==============================] - 3s 282ms/step - loss: 1936807.0000 - mean_absolute_error: 1923537.7500 - val_loss: 1809200.6250 - val_mean_absolute_error: 1796145.8750
Epoch 6/150
10/10 [==============================] - 2s 235ms/step

INFO:tensorflow:Assets written to: ./models/MobileNetV2/model/assets


INFO:tensorflow:Assets written to: ./models/MobileNetV2/model/assets


Evaluating Model...
6/6 [==============================] - 3s 343ms/step
Model Evaluation:
{'Timestamp': '2024-04-22 12:01:10.015847', 'R2': 0.3197749694068269, 'MAE': 1470665.3167177914, 'Percentage Error': 55.93252305824403, 'MSE': 4412585725390.41, 'Feature Importances': None}
Median Evaluation:
{'R2': 0.3113964044602101, 'MAE': 1485475.259202454, 'Percentage Error': 55.91313852380283, 'MSE': 4466937056817.391}
Done!


# Running on CPU

In [5]:
%reload_ext autoreload
%autoreload 2
MODELS_PATH: str = "./models"
USE_GPU: bool = False

### RF

In [58]:
from models import RF
TYPE = "RF"
MODEL_NAME: str = "RF"
FUNCTION: object = RF
ARGS: tuple = (
    train2_features,
    train2_prices,
)
train_save_model(FUNCTION, ARGS, test_images, test_features, test_prices, f"{MODELS_PATH}/{MODEL_NAME}", USE_GPU, TYPE)

Setting CPU
Saving Model...
Evaluating Model...

Model Evaluation:
{'Timestamp': '2024-04-24 14:57:43.213742', 'R2': 0.8289665942093328, 'MAE': 996231.6707605342, 'Percentage Error': 28.018551978180028, 'MSE': 1984068300094.8555, 'Feature Importances': {'lattitude': 0.01602962256428797, 'longitude': 0.042573922319192915, 'postal_code': 0.11869857747103998, 'type': 0.0, 'postal_avg_sqm_price': 0.0, 'size': 0.570671807578891, 'basement_size': 0.0, 'rooms': 0.22731129682279755, 'year_built': 0.012148670466301418, 'year_rebuilt': 0.009125838054946411, 'energy_label': 0.003440264722542739}}

Median Evaluation:
{'R2': 0.8241391743510718, 'MAE': 1004422.6693959701, 'Percentage Error': 27.814622804485744, 'MSE': 2040068650831.8735}
Feauter Importance...
{'lattitude': 0.01602962256428797, 'longitude': 0.042573922319192915, 'postal_code': 0.11869857747103998, 'type': 0.0, 'postal_avg_sqm_price': 0.0, 'size': 0.570671807578891, 'basement_size': 0.0, 'rooms': 0.22731129682279755, 'year_built': 0.0

### CNN

In [89]:
from models import CNN_model
TYPE = 'CNN'
MODEL_NAME: str = "MobileNetV2"
FUNCTION: object = CNN_model
ARGS: tuple = (
    MobileNetV3Small,
    True,
    train1_images,
    train1_prices,
    valid_images,
    valid_prices,
)
train_save_model(FUNCTION, ARGS, test_images, test_features, test_prices, f"{MODELS_PATH}/{MODEL_NAME}", USE_GPU, TYPE)

Setting CPU


Compiling Model
Fitting Model
4/4 [==============================] - 35s 4s/step - loss: 3978814.5000 - mean_absolute_error: 3977173.2500 - val_loss: 3794452.5000 - val_mean_absolute_error: 3793638.5000
Saving Model...
INFO:tensorflow:Assets written to: ./models/MobileNetV2/model/assets


INFO:tensorflow:Assets written to: ./models/MobileNetV2/model/assets


2/2 [==============================] - 11s 2s/step


AttributeError: 'Image' object has no attribute 'show'

### CNN RF

In [37]:
img_model =  keras.models.load_model(f"{MODELS_PATH}/MobileNetV2/model")

In [45]:
from models import CNN_RF_model
TYPE = 'CNN_RF'
MODEL_NAME: str = "MobileNetV2_RF"
FUNCTION: object = CNN_RF_model
ARGS: tuple = (
    img_model, # keras.models.load_model(f"{MODELS_PATH}/MobileNetV2/model")
    train2_images,
    train2_features,
    train2_prices,
)
train_save_model(FUNCTION, ARGS, test_images, test_features,  test_prices, f"{MODELS_PATH}/{MODEL_NAME}", USE_GPU, TYPE)

Setting CPU
4/4 [==============================] - 15s 3s/step
Saving Model...
2/2 [==============================] - 9s 4s/step
Evaluating Model...
Model Evaluation:
{'Timestamp': '2024-04-24 14:35:53.246289', 'R2': 0.8160100811706473, 'MAE': 1017904.6852259422, 'Percentage Error': 28.601578353487096, 'MSE': 2134369971753.5764, 'Feature Importances': {'image_predictions': 0.04981168165048342, 'lattitude': 0.012466178841286406, 'longitude': 0.03442733501581914, 'postal_code': 0.11451325660153887, 'type': 0.0, 'postal_avg_sqm_price': 0.0, 'size': 0.5968602239710372, 'basement_size': 0.0, 'rooms': 0.16058157954733135, 'year_built': 0.019117720348834296, 'year_rebuilt': 0.009714347681800023, 'energy_label': 0.0025076763418691965}}
Median Evaluation:
{'R2': 0.8218716651945884, 'MAE': 1011506.9829500999, 'Percentage Error': 28.022329718048578, 'MSE': 2066372828175.2158}
Feauter Importance...
{'image_predictions': 0.04981168165048342, 'lattitude': 0.012466178841286406, 'longitude': 0.0344273

/usr/local/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Saving Feature Importance
Done!


### CN_AE_RF

In [40]:
from models import CNN_AE_RF_model
#img_model =  keras.models.load_model(f"{MODELS_PATH}/MobileNetV2/model")
TYPE = 'CNN_RF'
MODEL_NAME: str = "MobileNetV2_AE_RF"
FUNCTION: object = CNN_AE_RF_model
ARGS: tuple = (
    img_model, #keras.models.load_model(f"{MODELS_PATH}/MobileNetV2/model")
    train2_images,
    train2_features,
    train2_prices,
)
train_save_model(FUNCTION, ARGS, test_images, test_features,  test_prices, f"{MODELS_PATH}/{MODEL_NAME}", USE_GPU, TYPE)

Setting CPU
Epoch 1/10
4/4 [==============================] - 17s 3s/step - loss: 0.2045 - val_loss: 0.2089
Epoch 2/10
4/4 [==============================] - 12s 3s/step - loss: 0.1961 - val_loss: 0.1978
Epoch 3/10
4/4 [==============================] - 11s 3s/step - loss: 0.1841 - val_loss: 0.1791
Epoch 4/10
4/4 [==============================] - 12s 3s/step - loss: 0.1638 - val_loss: 0.1497
Epoch 5/10
4/4 [==============================] - 12s 3s/step - loss: 0.1335 - val_loss: 0.1100
Epoch 6/10
4/4 [==============================] - 12s 3s/step - loss: 0.0952 - val_loss: 0.0696
Epoch 7/10
4/4 [==============================] - 12s 3s/step - loss: 0.0611 - val_loss: 0.0456
Epoch 8/10
4/4 [==============================] - 11s 3s/step - loss: 0.0455 - val_loss: 0.0402
Epoch 9/10
4/4 [==============================] - 11s 3s/step - loss: 0.0441 - val_loss: 0.0404
Epoch 10/10
4/4 [==============================] - 11s 2s/step
Saving Model...
2/2 [==============================] - 7s 3s/

/usr/local/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Evaluating Model...
Model Evaluation:
{'Timestamp': '2024-04-24 14:22:38.386143', 'R2': 0.8511217609475337, 'MAE': 970361.2261451767, 'Percentage Error': 29.046820972564607, 'MSE': 1727057900252.9617, 'Feature Importances': {'image_predictions': 0.0463097610424511, 'reconstruction_error': 0.04876594116836159, 'lattitude': 0.01587018442779833, 'longitude': 0.038786463741958474, 'postal_code': 0.10793556853748301, 'type': 0.0, 'postal_avg_sqm_price': 0.0, 'size': 0.5737296348626781, 'basement_size': 0.0, 'rooms': 0.1501734766219969, 'year_built': 0.0073559505727489646, 'year_rebuilt': 0.008787498384729194, 'energy_label': 0.002285520639794364}}
Median Evaluation:
{'R2': 0.8604386880810646, 'MAE': 952247.6402482346, 'Percentage Error': 27.988216235197285, 'MSE': 1618977144365.0898}
Saving Prediction Results
Saving Feature Importance
Done!


### N-CNN

In [84]:
from models import N_CNN_RF_model
from keras.applications import MobileNetV3Small
TYPE = 'CNN_RF'
MODEL_NAME: str = "N_CNN_MobileNetV2_RF"
FUNCTION: object = N_CNN_RF_model
ARGS: tuple = (
    2,
    MobileNetV3Small,
    np.concatenate((train1_images, train2_images), axis=0),
    pd.concat((train1_features, train2_features), axis=0),

    np.concatenate((train1_prices, train2_prices), axis=0),
)
train_save_model(FUNCTION, ARGS, test_images, test_features,  test_prices, f"{MODELS_PATH}/{MODEL_NAME}", USE_GPU, TYPE)

Setting CPU


Compiling Model
Fitting Model
4/4 [==============================] - 48s 6s/step - loss: 4100226.2500 - mean_absolute_error: 4098585.0000 - val_loss: 3974956.7500 - val_mean_absolute_error: 3974130.2500


Compiling Model
Fitting Model
7/7 [==============================] - 16s 2s/step
Saving Model...
2/2 [==============================] - 4s 2s/step
Evaluating Model...

Model Evaluation:
{'Timestamp': '2024-04-24 16:12:42.383736', 'R2': 0.8319390956849861, 'MAE': 941311.0689502058, 'Percentage Error': 26.393382586127913, 'MSE': 1949585878824.2, 'Feature Importances': {'image_predictions': 0.04670235477127162, 'lattitude': 0.01527586527232909, 'longitude': 0.03158211906832811, 'postal_code': 0.09247071796454477, 'type': 0.0, 'postal_avg_sqm_price': 0.0, 'size': 0.7465850811467064, 'basement_size': 0.0, 'rooms': 0.03574781906612901, 'year_built': 0.015633103547971344, 'year_rebuilt': 0.011698572992880725, 'energy_label': 0.004304366169838972}}

Median Evaluation:
{'R2': 0.8319390956849861, 'MAE': 941311.0689502058, 'Percentage Error': 26.393382586127913, 'MSE': 1949585878824.2}
Feauter Importance...
{'image_predictions': 0.04670235477127162, 'lattitude': 0.01527586527232909, 'longitude': 

/usr/local/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(



Saving Feature Importance

Done!
